In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("Fake_and_Real_News").getOrCreate()

In [4]:
train = spark.read.parquet("../data/face-and-real-news-dataset/train.parquet")

In [5]:
test = spark.read.parquet("../data/face-and-real-news-dataset/test.parquet")

In [6]:
from pyspark.ml.feature import VectorAssembler

In [7]:
# assembler = VectorAssembler(inputCols=['title_text_idf', 'subject_idx'], outputCol='features')
assembler = VectorAssembler(inputCols=['text_idf', 'subject_idx'], outputCol='features')

In [8]:
train = assembler.transform(train).select('label', 'features')

In [9]:
test = assembler.transform(test).select('label', 'features')

# 7. Build model
## 7.1. Logistic model
### 7.1.1. Build model

In [10]:
from pyspark.ml.classification import NaiveBayes

In [11]:
nb = NaiveBayes()

In [12]:
nv_model_0 = nb.fit(train)

In [13]:
prediction_0 = nv_model_0.transform(test)

### 7.1.2. Đánh giá model
####

In [14]:
prediction_0.groupBy('label', 'prediction').count().show()

Py4JJavaError: An error occurred while calling o127.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 4.0 failed 1 times, most recent failure: Lost task 7.0 in stage 4.0 (TID 221) (192.168.1.10 executor driver): org.apache.spark.SparkException: Failed to execute user defined function(ProbabilisticClassificationModel$$Lambda$3539/0x0000000841144840: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.agg_doAggregateWithKeys_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:75

In [15]:
prediction_0.show()

Py4JJavaError: An error occurred while calling o122.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 6.0 failed 1 times, most recent failure: Lost task 0.0 in stage 6.0 (TID 226) (192.168.1.10 executor driver): org.apache.spark.SparkException: Failed to execute user defined function(ProbabilisticClassificationModel$$Lambda$3485/0x0000000841279040: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:755)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$getByteArrayRdd$1(SparkPlan.scala:345)
	at org.apache.spark.rdd.RDD.$anonfun$mapP

In [29]:
type(prediction_0)

pyspark.sql.dataframe.DataFrame